# Binomial Tree Function

In [1]:
# Function to price an American put using a Binomial Tree
binomial_tree_american_put <- function(S0, K, T, r, sigma, M) {
  dt <- T / M  #Time Step Size 
  u <- exp(sigma * sqrt(dt)) # Upturn factor
  d <- exp(-sigma * sqrt(dt)) # Downward factor
  p <- (exp(r * dt) - d) / (u - d) # Risk-neutral probability

  # Stock price tree
  stock_tree <- matrix(0, nrow = M + 1, ncol = M + 1) # Create matrix to store stock prices
  for (j in 0:M) {
    for (i in 0:j) {
      stock_tree[i + 1, j + 1] <- S0 * u^i * d^(j - i) # Number of steps multiplied by upturn and downturn factors
    }
  }
  # Option value tree
  option_tree <- matrix(0, nrow = M + 1, ncol = M + 1) # Matrix to store option values 
  option_tree[, M + 1] <- pmax(K - stock_tree[, M + 1], 0)  # Payoff is max(K - S, 0) at maturity, applied to entire column M+1

  # Work backward through the tree
  for (j in (M - 1):0) { # Go through each column in reverse order starting at M-1  
    for (i in 0:j) { # Go though each row in the column
      continuation <- exp(-r * dt) * (p * option_tree[i + 2, j + 2] + (1 - p) * option_tree[i + 1, j + 2])
      exercise <- pmax(K - stock_tree[i + 1, j + 1], 0)
      option_tree[i + 1, j + 1] <- pmax(continuation, exercise)
    }
  }
  
  return(option_tree[1, 1])  # Option price at root
}

In [2]:
# Test with example from class 
binomial_tree_american_put(S0=4, K=3.5, T=1.5, r=0.05, sigma=0.2, M=3) # Gives same result

[1] 0.09751714

In [3]:
# Parameters for Initial Testing 
S0 <- 931.8 # Inital stock price of SNP500 from estimation
r <- 0.005290473 # Risk-free rate from estimation 
sigma <- 0.01026789 # Volatility from estimation
M <- 1000  # Time steps for binomial tree
T <- 1

In [4]:
# Example 1 comparion with Monte Carlo, slightly in the money  
K <- 940
binomial_tree_american_put(S0, K, T, r, sigma, M)

[1] 8.2

In [5]:
# Example 2 comparison with Monte Carlo, at the money strike price
K <- 931.8
binomial_tree_american_put(S0, K, T, r, sigma, M)

[1] 2.323929

In [6]:
# Example 3 comparison with Monte Carlo, very deep out of the money strike price
K <- 929
binomial_tree_american_put(S0, K, T, r, sigma, M)

[1] 1.376628

In [7]:
# Example 4 comparison with Monte Carlo, very deep in the money strike price
K <- 970
binomial_tree_american_put(S0, K, T, r, sigma, M)

[1] 38.2